In [18]:
import pickle 
import pandas as pd
  
# Load the model from file 
nn_model = pickle.load((open("./pickled_files/nn.pkl", "rb")))
tfidf = pickle.load((open("./pickled_files/tfidf.pkl", "rb")))
#Load the dataframe from file
df = pd.read_csv('./pickled_files/cannabis.csv')

In [19]:
dtm = tfidf.fit_transform(df['General_Description'])
general_dtm = pd.DataFrame(dtm.todense(), columns=tfidf.get_feature_names())
nn_model = nn_model.fit(general_dtm)
df.head()

,Unnamed: 0,index,Strain,Type,Rating,Effects,Flavor,Description,Effects_tokens,Flavor_tokens,Description_tokens,General_Description
0,0,1,98-White-Widow,hybrid,4.7,"Relaxed,Aroused,Creative,Happy,Energetic","Flowery,Violet,Diesel",The ‘98 Aloha White Widow is an especially pot...,"['relaxed', 'aroused', 'creative', 'happy', 'e...","['flowery', 'violet', 'diesel']","['‘98', 'aloha', 'white', 'widow', 'especially...","Relaxed,Aroused,Creative,Happy,EnergeticFlower..."
1,1,2,1024,sativa,4.4,"Uplifted,Happy,Relaxed,Energetic,Creative","Spicy/Herbal,Sage,Woody",1024 is a sativa-dominant hybrid bred in Spain...,"['uplifted', 'happy', 'relaxed', 'energetic', ...","['spicy', 'herbal', 'sage', 'woody']","['1024', 'sativa-dominant', 'hybrid', 'bred', ...","Uplifted,Happy,Relaxed,Energetic,CreativeSpicy..."
2,2,3,13-Dawgs,hybrid,4.2,"Tingly,Creative,Hungry,Relaxed,Uplifted","Apricot,Citrus,Grapefruit",13 Dawgs is a hybrid of G13 and Chemdawg genet...,"['tingly', 'creative', 'hungry', 'relaxed', 'u...","['apricot', 'citrus', 'grapefruit']","['13', 'dawgs', 'hybrid', 'g13', 'chemdawg', '...","Tingly,Creative,Hungry,Relaxed,UpliftedApricot..."
3,3,4,24K-Gold,hybrid,4.6,"Happy,Relaxed,Euphoric,Uplifted,Talkative","Citrus,Earthy,Orange","Also known as Kosher Tangie, 24k Gold is a 60%...","['happy', 'relaxed', 'euphoric', 'uplifted', '...","['citrus', 'earthy', 'orange']","['known', 'kosher', 'tangie,', '24k', 'gold', ...","Happy,Relaxed,Euphoric,Uplifted,TalkativeCitru..."
4,4,6,3-Kings,hybrid,4.4,"Relaxed,Euphoric,Happy,Uplifted,Hungry","Earthy,Sweet,Pungent","The 3 Kings marijuana strain, a holy trinity o...","['relaxed', 'euphoric', 'happy', 'uplifted', '...","['earthy', 'sweet', 'pungent']","['3', 'kings', 'marijuana', 'strain,', 'holy',...","Relaxed,Euphoric,Happy,Uplifted,HungryEarthy,S..."


In [20]:
## GOING TO NEED TO CREATE A FUNCTION TO PARSE
## THE JSON DICTIONARY SENT TO US TO MATCH THE BELOW 
## PYTHON DICTIONARY.

user_inputs = {
        'effects': ['happy','creative'],
        'flavors': ['strawberry, pineapple'],
        'ailments': ['depression, headaches']
    }

def get_user_inputs(data):
    user_desc = ''
    for index in data:
        for values in data[index]:
            user_desc += '\n' + values
    return tfidf.transform([user_desc])
    
strain_query = get_user_inputs(user_inputs)

In [23]:
def output_user_reccomendations(query, dframe):
    _, similar_topic_indices = nn_model.kneighbors(query.todense())
    indices = list(similar_topic_indices[0])
    print(indices)
    
    recc_values = []
    
    def get_index_values(index):
        output_values = {
        'ID':[],
        'Strain':[], 
        'Type':[], 
        'Rating':[], 
        'Flavor':[], 
        'Effects':[], 
        'Description':[]}
        
        ##Updated output, commented out others just in case you want to change back.
        #output_values['Strain'].append(dframe['Strain'][index])
        output_values['ID'].append(index)
        #print(list(output_values.keys()))
        for value in list(output_values.keys())[1:]:
            output_values[value].append(dframe[value][index])
  
        return output_values

    for key in range(0, len(indices)):
        recc_values.append(get_index_values(indices[key]))
    
    return recc_values

reccomondations = output_user_reccomendations(strain_query, df)

[768, 1369, 1720, 1707, 1708]


In [24]:
reccomondations

[{'ID': [768],
  'Strain': ['Golden-Pineapple'],
  'Type': ['hybrid'],
  'Rating': [4.5],
  'Flavor': ['Pineapple,Tropical,Citrus'],
  'Effects': ['Happy,Euphoric,Uplifted,Relaxed,Creative'],
  'Description': ['Golden Pineapple is a hybrid cross between Golden Goat and Pineapple Kush that delivers creative, uplifting effects with a fruity, tropical flavor. Its aroma is remarkably similar to sour pineapple, providing a flavorful escape from stress, anxiety, and depression. Golden Pineapple’s engaged, active effects will give you the energy you need to keep going throughout your day, although in larger doses, it can be difficult to direct that focus effectively.']},
 {'ID': [1369],
  'Strain': ['Pineapple-Og'],
  'Type': ['sativa'],
  'Rating': [4.6],
  'Flavor': ['Pineapple,Citrus,Tropical'],
  'Effects': ['Happy,Uplifted,Focused,Creative,Energetic'],
  'Description': ['This 70% sativa hybrid is thought to be a cross between Pineapple Express and an unknown OG Kush hybrid, and is probab

In [25]:
### NEED TO CREATE A FUNCTION TO CONVERT THE RECOMMONDATIONS
### INTO A JSON DICTIONARY

In [26]:
df.head()

,Unnamed: 0,index,Strain,Type,Rating,Effects,Flavor,Description,Effects_tokens,Flavor_tokens,Description_tokens,General_Description
0,0,1,98-White-Widow,hybrid,4.7,"Relaxed,Aroused,Creative,Happy,Energetic","Flowery,Violet,Diesel",The ‘98 Aloha White Widow is an especially pot...,"['relaxed', 'aroused', 'creative', 'happy', 'e...","['flowery', 'violet', 'diesel']","['‘98', 'aloha', 'white', 'widow', 'especially...","Relaxed,Aroused,Creative,Happy,EnergeticFlower..."
1,1,2,1024,sativa,4.4,"Uplifted,Happy,Relaxed,Energetic,Creative","Spicy/Herbal,Sage,Woody",1024 is a sativa-dominant hybrid bred in Spain...,"['uplifted', 'happy', 'relaxed', 'energetic', ...","['spicy', 'herbal', 'sage', 'woody']","['1024', 'sativa-dominant', 'hybrid', 'bred', ...","Uplifted,Happy,Relaxed,Energetic,CreativeSpicy..."
2,2,3,13-Dawgs,hybrid,4.2,"Tingly,Creative,Hungry,Relaxed,Uplifted","Apricot,Citrus,Grapefruit",13 Dawgs is a hybrid of G13 and Chemdawg genet...,"['tingly', 'creative', 'hungry', 'relaxed', 'u...","['apricot', 'citrus', 'grapefruit']","['13', 'dawgs', 'hybrid', 'g13', 'chemdawg', '...","Tingly,Creative,Hungry,Relaxed,UpliftedApricot..."
3,3,4,24K-Gold,hybrid,4.6,"Happy,Relaxed,Euphoric,Uplifted,Talkative","Citrus,Earthy,Orange","Also known as Kosher Tangie, 24k Gold is a 60%...","['happy', 'relaxed', 'euphoric', 'uplifted', '...","['citrus', 'earthy', 'orange']","['known', 'kosher', 'tangie,', '24k', 'gold', ...","Happy,Relaxed,Euphoric,Uplifted,TalkativeCitru..."
4,4,6,3-Kings,hybrid,4.4,"Relaxed,Euphoric,Happy,Uplifted,Hungry","Earthy,Sweet,Pungent","The 3 Kings marijuana strain, a holy trinity o...","['relaxed', 'euphoric', 'happy', 'uplifted', '...","['earthy', 'sweet', 'pungent']","['3', 'kings', 'marijuana', 'strain,', 'holy',...","Relaxed,Euphoric,Happy,Uplifted,HungryEarthy,S..."
